In [48]:
def table_to_output(groupby_table):
    temp = groupby_table.reset_index()
    temp2 = temp[temp['True_Label']=="Phage"]
    if len(temp2)==0:
        False_Negative_Phage = Phage_Count
    else:
        Phage_Predicted = temp2.iloc[0][1]
        False_Negative_Phage = Phage_Count - Phage_Predicted
        
    
    temp3 = temp[temp['True_Label']=="Prophage"]
    if len(temp3)==0:
        False_Negative_Prophage = Prophage_Count
    else:
        Prophage_Predicted = temp3.iloc[0][1]
        False_Negative_Prophage = Prophage_Count - Prophage_Predicted
 
    return(False_Negative_Phage, False_Negative_Prophage)

def Score(FNP, FNPro, data, Prophage=True):
    if Prophage==True:
        y_true = [1 if x == "Phage" else 0 for x in data['True_Label']]
        y_pred = [1]*len(data)

        listofzeros = [0]*FNP
        listofones = [1]*FNP

        y_pred = y_pred + listofzeros
        y_true = y_true + listofones

    else:
        data['True_Label'] = data['True_Label'].replace(regex='Prophage', value="Phage")
        y_true = [1 if x == "Phage" else 0 for x in data['True_Label']]
        y_pred = [1]*len(data)
        listofzeros = [0]*FNP
        listofones = [1]*FNP

        y_pred = y_pred + listofzeros
        y_true = y_true + listofones

        listofzeros2 = [0]*FNPro
        listofones2 = [1]*FNPro

        y_pred = y_pred + listofzeros2
        y_true = y_true + listofones2

        
    p = precision_score(y_true, y_pred)
    r = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    return(p, r, f1)

In [72]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
input_file = "../../Kraken_Output/ZHC.csv"
df = pd.read_csv(input_file)

simulation = 'Zero'
complexity = 'High'
df2 = pd.read_csv("../../Contig_Abundance_By_Class_Simulation.csv")

df1 = df2[(df2.Simulation == simulation) & (df2.Complexity == complexity)]
try:
    Phage_Count = int(df1['Phage'])
except:
    Phage_Count = 0
try:
    Prophage_Count = int(df1['Prophage'])
except:
    Prophage_Count = 0

df['True_Label'] = [x.split("_")[0] for x in df["ContigID"]]

In [73]:
e = df.groupby("True_Label").count()
FNP, FNPro = table_to_output(e)
p, r, f1 = Score(FNP, FNPro, df, Prophage=True)

my_dict = {}
my_dict['Prophage_Bacteria'] = [p,r,f1]


p, r, f1 = Score(FNP, FNPro, df, Prophage=False)
my_dict['Prophage_Viruses'] = [p,r,f1]

w = pd.DataFrame(my_dict.items(), columns=["Prophage Category",2])
w = w.set_index("Prophage Category")
w['Precision'] = [x[0] for x in w[2]]
w['Recall'] = [x[1] for x in w[2]]
w['F1 Score'] = [x[2] for x in w[2]]

w['Simulation'] = "zero"
w['Complexity'] = "high"
w['Tool'] = "Kraken"
w['Parameter'] ="NA"

w = w.drop(2, axis=1)
w.to_csv("../../Kraken_Output/Kraken_Scores.csv", header=None, mode='a')

In [74]:
## Remove Prophages
def table_to_output(groupby_table):
    temp = groupby_table.reset_index()
    temp2 = temp[temp['True_Label']=="Phage"]
    if len(temp2)==0:
        False_Negative_Phage = Phage_Count
    else:
        Phage_Predicted = temp2.iloc[0][1]
        False_Negative_Phage = Phage_Count - Phage_Predicted
 
    return(False_Negative_Phage)

def Score(FNP, data):
    y_true = [1 if x == "Phage" else 0 for x in data['True_Label']]
    y_pred = [1]*len(data)

    listofzeros = [0]*FNP
    listofones = [1]*FNP

    y_pred = y_pred + listofzeros
    y_true = y_true + listofones
     
    p = precision_score(y_true, y_pred)
    r = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    return(p, r, f1)

In [118]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
input_file = "../../Kraken_Output/ELC.csv"
df = pd.read_csv(input_file)

simulation = 'Exponential'
complexity = 'Low'
df2 = pd.read_csv("../../Contig_Abundance_By_Class_Simulation.csv")

df1 = df2[(df2.Simulation == simulation) & (df2.Complexity == complexity)]
try:
    Phage_Count = int(df1['Phage'])
except:
    Phage_Count = 0


df['True_Label'] = [x.split("_")[0] for x in df["ContigID"]]

In [119]:
e = df.groupby("True_Label").count()
FNP = table_to_output(e)
my_dict = {}
p, r, f1 = Score(FNP, df)
my_dict['Prophage_Removed'] = [p,r,f1]

w = pd.DataFrame(my_dict.items(), columns=["Prophage Category",2])
w = w.set_index("Prophage Category")
w['Precision'] = [x[0] for x in w[2]]
w['Recall'] = [x[1] for x in w[2]]
w['F1 Score'] = [x[2] for x in w[2]]

w['Simulation'] = "exponential"
w['Complexity'] = "low"
w['Tool'] = "Kraken"
w['Parameter'] ="NA"

w = w.drop(2, axis=1)
w.to_csv("../../Kraken_Output/Kraken_Scores_Prophage_Removed.csv", header=None, mode='a')